In [24]:
import pandas as pd
import wikipedia as wp
import re
import warnings
warnings.filterwarnings('ignore')

In [25]:
url = 'https://en.wikipedia.org/wiki/Telangana_Legislative_Assembly_election,_2014'
#I was playing with BeautifulSoup earlier, so I knew it was at that location.
df = pd.read_html(url)[5]
df1=pd.DataFrame(df)

#### The table has hierarchical indexing , which means  , it neeeds some cleaning.

In [26]:
#whats inside my "df1". That doesn't look good at all.
df1.head()
# let me drop the column that says district on it's head.
df1_new=df1.drop(0,axis=1)
df1_new.head()

,1,2,3,4
0,S. No.,Assembly Constituency,Winner,Party
1,1,Sirpur,KONERU KONAPPA,BSP
2,Chennur (SC),Nallala Odelu,INC,NaN
3,Bellampalli (SC),DURGAM CHINNAIAH,CPL,NaN
4,Mancheriala,DIWAKAR RAO NADIPELLI,INC,NaN


In [27]:
#So,its just that the table is a lil' off the row indexing.Alls I have to do is meove them to left.
df1_new.loc[[1,2,3]]= df1_new.loc[[1,2,3]].shift(-1,axis=0) 
df1_new.head()

,1,2,3,4
0,S. No.,Assembly Constituency,Winner,Party
1,Chennur (SC),Nallala Odelu,INC,NaN
2,Bellampalli (SC),DURGAM CHINNAIAH,CPL,NaN
3,NaN,NaN,NaN,NaN
4,Mancheriala,DIWAKAR RAO NADIPELLI,INC,NaN


In [28]:
#Use regex to find the rows with bad values.
first_col=df1_new[1]
bad_rows = (first_col.str.extract('(\d+)'))
first_col[first_col==bad_rows]

77      72
91      86
103     98
115    110
Name: 1, dtype: object

In [29]:
# Thats it, I got the indexes of rows with the bad data.Alls left 
# to do , just gut 'em out.
bad_row_nums = [77,91,103,115]
bad_row_df=df1_new.iloc[bad_row_nums]
bad_row_df.loc[:,[1,2,3,4]]= bad_row_df.loc[:,[1,2,3,4]].shift(-1,axis=1)

#### My aha! moment happened when I realised the old ugly values are still sitting in there. Do it one more time and gut em out.Indexes have changed this time.

In [30]:
# Let's merge the new df with the good ol one earlier.
frames = [df1_new,bad_row_df]
result = pd.concat(frames)
first_res = result[1]
ugly_cell=(first_res.str.extract('(\d+)'))
ugly_cell.value_counts()

110    1
86     1
98     1
72     1
Name: 1, dtype: int64

In [31]:
#Gutting the ugly values and dropping the Na values.
first_res[first_res == ugly_cell]
new_result=result.drop(result.index[[77,91,103,115]])
new_result=new_result.drop(4,axis=1)
#drop the NA's
clean_df=new_result.dropna(axis=0, how='all')


In [32]:
#convet the dataframe to a csv
clean_df.to_csv('mlas_2014.csv',header=0,index=False)

In [33]:
mla_table=pd.read_csv("mlas_2014.csv")

In [34]:
mla_table.columns

Index(['S. No.', 'Assembly Constituency', 'Winner'], dtype='object')

In [35]:
#rename the columns
mla_table.rename(columns={
                 'S. No.': 'dist',
                 'Assembly Constituency': 'mla',
                 'Winner': 'party'}, inplace=True)



In [36]:
dist_list = mla_table["dist"]

In [37]:
paren_lc=(re.sub(r'\([^)]*\)', '', row) for row in dist_list)

In [17]:
mla_table

,dist,mla,party
0,Chennur (SC),Nallala Odelu,INC
1,Bellampalli (SC),DURGAM CHINNAIAH,CPL
2,Mancheriala,DIWAKAR RAO NADIPELLI,INC
3,Asifabad (ST),KOVA LAXMI,INC
4,Khanapur (ST),AJMEERA REKHA,TDP
5,Adilabad,Jogu Ramanna,TRS
6,Boath (ST),RATHOD BAPURAO,INC
7,Nirmal,ALLOLLA INDRA KARAN REDDY,BSP
8,Mudhole,GADDIGARI VITTAL REDDY,INC
9,Bodhan,SHAKIL,TRS
